# Import dependencies

In [ ]:
import os
import sys
import pandas as pd

# Import library with current code functions
sys.path.append(os.path.join("..", "lib"))
import manual_labeler_functions as man_lab_fun, automatic_labeler_functions as aut_lab_fun, general_functions as gf, files_paths as fp

## Getting the list of measure files to be labeled and the list of seed available

In [ ]:
FILE_LIST_VD_MEASURE = gf.find_files_in_all_subdirectories([fp.DATASET_YT, fp.DATASET_LOCAL], fp.VD_MEASURE_L0)
FILE_LIST_SEED = gf.find_files_in_all_subdirectories([fp.DATASET_SEED], fp.VD_LABELED_L0)

In [ ]:
FILE_LIST_SEED

## Remove all VD_LABELED_L0 file

In [ ]:
def remove_file(FILE_LIST_PATH, file_name):
    for current_path in FILE_LIST_PATH:
        vd_labeled_path = os.path.join(os.path.dirname(current_path), file_name)
        if os.path.exists(vd_labeled_path):
            os.remove(vd_labeled_path)
            print(f"The file was removed: {vd_labeled_path}")
        else:
            print("File not found.")

In [ ]:
file_name = fp.VD_LABELED_L0
remove_file(FILE_LIST_VD_MEASURE, file_name)

## Select reference SEED

In [ ]:
files_labeled = FILE_LIST_SEED[:]

## Plot query serie

In [ ]:
current_labeling_class = 0 # Change for the correspondent number of your labeling class
frame_distance_threshold = 10
euclidean_distance_threshold = 1
for labeled_file in files_labeled:
    RESUME_DT = pd.DataFrame()
    print(f'\n-------{os.path.basename(os.path.dirname(labeled_file))}--------\n')
    for i, current_path_location in enumerate(FILE_LIST_VD_MEASURE):
        #print (str(i+1) + " of " + str(len(FILE_LIST_VD_MEASURE)) + ": " + current_path_location)

        path_dir = os.path.dirname(labeled_file)
        vd_labeled_path = os.path.join(path_dir, fp.VD_LABELED_L0)
        vd_labeled = pd.read_csv(vd_labeled_path)
        vd_labeled.drop(columns=['Unnamed: 0'], inplace=True)

        all_class = man_lab_fun.GET_ALL_CLASSES(vd_labeled)
        label_name = all_class[current_labeling_class]

        reference_measures = man_lab_fun.GET_MEASURES_FROM_CLASS(vd_labeled, label_name)

        frames = man_lab_fun.GET_FRAMES_FROM_CLASS(vd_labeled, label_name)

        all_measures_in_frame_interval = vd_labeled.loc[frames[:28]]
        selected_measures_in_frame_interval = all_measures_in_frame_interval[reference_measures]

        dict_label_parameters = {'label_name': label_name, 'reference_measures': reference_measures}

        RESUME_DT = aut_lab_fun.label_current_series(current_path_location, RESUME_DT, selected_measures_in_frame_interval, dict_label_parameters, os.path.dirname(labeled_file), distance_threshold=euclidian_distance_threshold, frame_threshold=frame_distance_threshold)  

    final_sum = RESUME_DT['final'].sum()
    print(f'Total of saved Subseries == {final_sum}') 